In [ ]:
#
# Attempts to implement the following:
#
# H. Rave, V. Molchanov and L. Linsen, "Uniform Sample Distribution in Scatterplots via Sector-based Transformation," 
# 2024 IEEE Visualization and Visual Analytics (VIS), St. Pete Beach, FL, USA, 2024, pp. 156-160, 
# doi: 10.1109/VIS55277.2024.00039. 
# keywords: {Data analysis;Visual analytics;Clutter;Scatterplot de-cluttering;spatial transformation},
#
import polars as pl
import numpy as np
from math import cos, sin, pi, sqrt, atan2
from shapely import Polygon
from   udist_scatterplots_via_sectors import UDistScatterPlotsViaSectors, xyUniformSampleDistributionSectorTransformDEBUG, xyUniformSampleDistributionSectorTransform
import time
import rtsvg
rt = rtsvg.RACETrack()
import random
num_of_pts   = [100, 200, 50]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

_iterations_ = 4

x_new, y_new, svgs, svgs_for_sectors, svg_animation, df_fine = xyUniformSampleDistributionSectorTransformDEBUG(rt, _xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_)

t0 = time.time()
xyUniformSampleDistributionSectorTransform(rt, _xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_)
t1 = time.time()
udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=_iterations_, debug=True)
t2 = time.time()
#
# Comparison (w/ fixed implementation)
# 351 Points (128 Iterations), 18.74s Reference Time, 1.66s Polars Time | (M1 Pro 16G) 
# 351 Points (256 Iterations), 38.45s Reference Time, 7.28s Polars Time | (M1 Pro 16G) 
#
print(f'{len(_xvals_)} Points ({_iterations_} Iterations), {t1-t0:.2f}s Reference Time, {t2-t1:.2f}s Polars Time')
rt.tile([svg_animation, udspvs.animateIterations(animation_dur="4s")], spacer=10)

In [ ]:
# 2048 Iterations w/ 351 Points | M1 Pro 16G
#
#                 sector_sums | 3.959
#                 arctangents | 2.436
#   ray_segment_intersections | 2.289
#              explode_points | 2.251
#         add_missing_sectors | 1.839
#                   area_calc | 1.254
#         sector_uv_summation | 0.907
#                point_update | 0.784
#       prepare_sector_angles | 0.659
#          join_sector_angles | 0.592
#                 all_sectors | 0.421
#                   normalize | 0.269
#                  prepare_df | 0.000
udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, _weights_, _colors_, iterations=16)
_lu_ = {'subroutine':[], 'time':[]}
for k in udspvs.time_lu: _lu_['subroutine'].append(k), _lu_['time'].append(udspvs.time_lu[k])
_df_ = pl.DataFrame(_lu_).sort(by='time', descending=True)
for i in range(len(_df_['subroutine'])): print(f'{_df_["subroutine"][i]:>28} | {_df_["time"][i]:.3f}')

In [ ]:
import copy
timing   = {'pts':[], 'iters':[], 'time':[]}
time_lus = []
for num_pts in [10,20,30]: # [1000, 1500, 2000]:
    for num_iter in [4, 8, 12]: # [16, 32, 64, 128, 256]:
        _xvals_, _yvals_ = [], []
        for i in range(num_pts):
            x, y = random.random() * 100, random.random() * 100
            _xvals_.append(x), _yvals_.append(y)
        t0 = time.time()
        udspvs = UDistScatterPlotsViaSectors(_xvals_, _yvals_, iterations=num_iter)
        t1 = time.time()
        print(f'{num_pts:>5} Pts | {num_iter:>3} Iters | {t1-t0:.2f}s')
        timing['pts'].append(num_pts), timing['iters'].append(num_iter), timing['time'].append(t1-t0)
        time_lus.append(copy.deepcopy(udspvs.time_lu))
#rt.xy(pl.DataFrame(timing), x_field='iters', y_field='time', line_groupby_field='pts', color_by='pts') #._repr_svg_()

'<svg id="xy_13657337" x="0" y="0" width="256" height="256" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="255" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 239 L 17 3 L 251 3 L 251 239 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,237 32,233 63,226 126,213 251,185 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,231 32,223 63,207 126,174 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,225 32,209 63,181 126,123 251,3 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="32" cy="209" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="181" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="185" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="231" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="223" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="226" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="233" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="3" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="174" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="237" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="213" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="225" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="207" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="123" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="253" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="253" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="253" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="237" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,237)">0.5</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">36.0</text><text x="13" text-anchor="middle" y="120.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,120.0)">time</text><rect width="255" height="256" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg>'

In [ ]:
#
# performance issues (in order): sector_sums ... then explode_points ... then arctangents
#
_lu_ = {'pts':[], 'iters':[], 'subroutine':[], 'subroutine_time':[]}
for i in range(len(time_lus)):
    _time_lu_               = time_lus[i]
    _pts_, _iters_, _times_ = timing['pts'][i], timing['iters'][i], timing['time'][i]
    for k in _time_lu_:
        _lu_['pts'].append(_pts_), _lu_['iters'].append(_iters_), _lu_['subroutine'].append(k), _lu_['subroutine_time'].append(_time_lu_[k])

#rt.smallMultiples(pl.DataFrame(_lu_), category_by='subroutine', sm_type='xy', y_axis_independent=False,
#                  sm_params={'x_field':'iters', 'y_field':'subroutine_time', 'line_groupby_field':'pts', 'color_by':'pts'},
#                  sort_by='field', sort_by_field='subroutine_time',
#                  h_sm_override=128, w_sm_override=256, w=1300) # ._repr_svg_()

'<svg id="smallMultiples_21023" x="0" y="0" width="1292" height="438" xmlns="http://www.w3.org/2000/svg"><rect width="1291" height="437" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><svg id="smallMultiples_21023_0" x="2" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,105 32,102 63,95 126,82 251,56 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,107 32,106 63,103 126,99 251,88 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,102 32,95 63,82 126,56 251,3 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="32" cy="95" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="99" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="106" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="3" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="103" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="102" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="82" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="56" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="107" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="56" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="88" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="105" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="95" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="102" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="82" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">sector_sums</text><svg id="smallMultiples_21023_1" x="260" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,106 32,103 63,97 126,85 251,59 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,107 126,105 251,101 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,107 32,105 63,102 126,94 251,80 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="32" cy="108" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="103" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="97" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="105" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="106" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="80" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="107" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="105" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="85" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="101" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="94" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="107" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="59" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="102" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">explode_points</text><svg id="smallMultiples_21023_2" x="518" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,105 32,103 63,98 126,88 251,68 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,107 63,106 126,103 251,97 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,107 32,105 63,102 126,97 251,85 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="251" cy="85" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="88" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="97" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="68" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="103" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="103" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="107" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="105" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="98" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="102" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="97" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="106" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="107" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="105" r="2" fill="#4166ce" stroke="#4166ce" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">arctangents</text><svg id="smallMultiples_21023_3" x="776" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,105 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="251" cy="105" r="2" fill="#72b1b4" stroke="#72b1b4" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="106" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="107" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">add_missing_sectors</text><svg id="smallMultiples_21023_4" x="1034" y="2" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,107 251,106 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="106" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="107" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="142" font-family="Times" fill="#000000" font-size="14px">ray_segment_intersections</text><svg id="smallMultiples_21023_5" x="2" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#b3c79f" stroke-width="1" fill="none" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="107" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">area_calc</text><svg id="smallMultiples_21023_6" x="260" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,107 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="107" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">sector_uv_summation</text><svg id="smallMultiples_21023_7" x="518" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">point_update</text><svg id="smallMultiples_21023_8" x="776" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">join_sector_angles</text><svg id="smallMultiples_21023_9" x="1034" y="148" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="288" font-family="Times" fill="#000000" font-size="14px">prepare_sector_angles</text><svg id="smallMultiples_21023_10" x="2" y="294" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="434" font-family="Times" fill="#000000" font-size="14px">all_sectors</text><svg id="smallMultiples_21023_11" x="260" y="294" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="434" font-family="Times" fill="#000000" font-size="14px">normalize</text><svg id="smallMultiples_21023_12" x="518" y="294" width="256" height="128" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="127" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 111 L 17 3 L 251 3 L 251 111 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 32,108 63,109 126,108 251,108 " stroke="#b3c79f" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#4166ce" stroke-width="1" fill="none" /><polyline points="17,108 32,108 63,108 126,108 251,108 " stroke="#72b1b4" stroke-width="1" fill="none" /><circle  cx="32" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="251" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="126" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="109" r="2" fill="#b3c79f" stroke="#b3c79f" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="63" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle  cx="17" cy="108" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="125" font-family="Times" fill="#000000" font-size="12px">16</text><text x="251" text-anchor="end" y="125" font-family="Times" fill="#000000" font-size="12px">256</text><text x="134.0" text-anchor="middle" y="125" font-family="Times" fill="#000000" font-size="12px">iters</text><text x="13" text-anchor="start" y="109" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,109)">0.0</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">18.3</text><rect width="255" height="128" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="434" font-family="Times" fill="#000000" font-size="14px">prepare_df</text><rect width="1291" height="437" x="0" y="0" fill="none" fill-opacity="0.0" stroke="#000000" /></svg>'

In [ ]:
#
# Prototype for Grouping Points into Tiles...
# ... then using those for sector_sums, explode_points, and arctangents
#
num_of_tiles = 64

#num_of_pts   = [100_000, 200_000, 50_000]
num_of_pts   = [1000, 2000, 500]
circle_geoms = [(5,5,1),(20,10,2),(8,8,1)]
colors       = ['#ff0000','#006400','#0000ff']
_xvals_, _yvals_, _weights_, _colors_ = [12.0], [8.0], [1.0], ['#000000']
for i in range(len(num_of_pts)):
    for j in range(num_of_pts[i]):
        a, l = random.random() * 2 * pi, random.random() * circle_geoms[i][2]
        x, y = circle_geoms[i][0] + l * cos(a), circle_geoms[i][1] + l * sin(a)
        _xvals_.append(x), _yvals_.append(y), _weights_.append(1.0), _colors_.append(colors[i])

df = pl.DataFrame({'x':_xvals_, 'y':_yvals_, 'w':_weights_, 'c':_colors_}).with_row_index('__index__')
df = df.with_columns((0.02 + 0.96 * (pl.col('x') - pl.col('x').min())/(pl.col('x').max() - pl.col('x').min())).alias('x'), 
                     (0.02 + 0.96 * (pl.col('y') - pl.col('y').min())/(pl.col('y').max() - pl.col('y').min())).alias('y'))

# Put into tiles -- xi, yi
t = time.time()
df = df.with_columns((pl.col('x') * num_of_tiles).cast(pl.Int16).alias('xi'), (pl.col('y') * num_of_tiles).cast(pl.Int16).alias('yi'))
t_tiling = time.time() - t

# Sum those tiles ... requires a new dataframe because we'll need the original points later on...
t = time.time()
df_tiled = df.group_by(['xi','yi']).agg(pl.col('w').sum().alias('_tile_sum_'))
t_tile_sums = time.time() - t

print(f'{t_tiling=:.3f} | {t_tile_sums=:.3f}')

df

In [ ]:
# tile_to_rect[(32,32)]
# (0.5, 0.5, 0.515625, 0.515625)
_render_tiles_, _render_points_, _render_rays_ = True, True, True
xpt, ypt = round(random.random(),3), round(random.random(),3)

svg = [f'<svg x="0" y="0" width="1024" height="1024" viewBox="0.0 0.0 1.0 1.0">']
svg.append(f'<rect x="0" y="0" width="1.0" height="1.0" fill="#ffffff"/>')

tile_to_rect = {}
if _render_tiles_:
    for xi in range(num_of_tiles):
        x0, x1 = xi/float(num_of_tiles), (xi+1)/float(num_of_tiles)
        for yi in range(num_of_tiles):
            y0, y1 = yi/float(num_of_tiles), (yi+1)/float(num_of_tiles)
            tile_to_rect[(xi,yi)] = (x0, y0, x1, y1)
            _color_   = rt.co_mgr.getColor(str((xi,yi)))
            svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="{_color_}" fill-opacity="0.5" />')

if _render_points_:
    for i in range(len(df)):
        x,y,xi,yi = df['x'][i], df['y'][i], df['xi'][i], df['yi'][i]
        _color_   = rt.co_mgr.getColor(str((xi,yi)))
        svg.append(f'<circle cx="{x}" cy="{y}" r="0.001" fill="{_color_}"/>')

tiles_intersected_by_rays = set()
if _render_rays_:
    for _sector_ in range(16):
        a0, a1         = _sector_*2*pi/16, (_sector_+1)*2*pi/16
        u0, v0, u1, v1 = cos(a0), sin(a0), cos(a1), sin(a1)
        svg.append(f'<line x1="{xpt}" y1="{ypt}" x2="{xpt+u0*2.0}" y2="{ypt+v0*2.0}" stroke="#000000" stroke-width="0.0001"/>')
        svg.append(f'<line x1="{xpt}" y1="{ypt}" x2="{xpt+u1*2.0}" y2="{ypt+v1*2.0}" stroke="#000000" stroke-width="0.0001"/>')
        for _tile_ in tile_to_rect:
            x0, y0, x1, y1 = tile_to_rect[_tile_]
            if rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y0), (x0, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x1, y0), (x1, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y0), (x1, y0)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u0, v0), (x0, y1), (x1, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y0), (x0, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x1, y0), (x1, y1)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y0), (x1, y0)) or \
               rt.rayIntersectsSegment((xpt, ypt), (u1, v1), (x0, y1), (x1, y1)): tiles_intersected_by_rays.add(_tile_)

svg.append('</svg>')

# num_of_tiles | total tiles | tiles intersected | xpt,   ypt
#           32 |        1024 |               338 | 0.48,  0.48
#           64 |        4096 |               711 | 0.48,  0.48
#           64 |        4096 |               562 | 0.728, 0.128
#           64 |        4096 |               446 | 0.044, 0.936
#           64 |        4096 |               625 | 0.465, 0.813
#           64 |        4096 |               548 | 0.716, 0.105
#
#




print(f'{num_of_tiles} | {len(tile_to_rect)} | {len(tiles_intersected_by_rays)} | {xpt}, {ypt}')

rt.tile([''.join(svg)])